#Collect Comments from Edmunds Sedan General Forum Discussion

We run the risk of undercounting mentions of brand if we ignore the model names. Want to catch BMW or 325, but don't want to double count when someone says both BMW and 325i. 

In [1]:
import re
import urllib2
from pandas import DataFrame, Series
import pandas as pd
from bs4 import BeautifulSoup
import time

In [36]:
url = 'http://forums.edmunds.com/discussions/tagged/x/sedans'

In [80]:
#this recurssive function gets links to the pages that have the links to the discussion threads
#The links to pages 1-n are displayed in the classic website structure: 1, 2, 3, 4, 5, 6 ... n >>>
def get_links_to_discussion_parent_pages(url1, count = 0):
    BASE_URL = 'http://forums.edmunds.com'
    page = urllib2.urlopen(url1).read()
    soup = BeautifulSoup(page, 'lxml')
    #check if there are any discussions posted on this page.
    are_there_discussions = re.findall('''<ul class="DataList Discussions"''', str(soup))
    if len(are_there_discussions) > 0:
        #If there are discussions, then function calls itself on the second to last page link
        Edmunds = soup.find("div", "PageControls Bottom")
        parent_links = []
        #save link to all and then open the second to last one
        time.sleep(1)
        count += 1
        if count > 32:
            print 'infinite loop'
            return ''
        pages = Edmunds.findAll("a")
        links = re.findall('''href=["'](.[^"']+)["']''', str(pages))
        if count == 1:
            for link in links[0:7]:
                parent_links.append(BASE_URL + link)
        else:
            for link in links[6:9]:
                parent_links.append(BASE_URL + link)
        parent_links.extend(get_links_to_discussion_parent_pages(BASE_URL + str(links[-4]), count = count))
        return parent_links
    else:
        return ''

In [81]:
links = get_links_to_discussion_parent_pages(url)
links

['http://forums.edmunds.com/discussions/tagged/x/sedans?Page=p1&amp;',
 'http://forums.edmunds.com/discussions/tagged/x/sedans?Page=p2&amp;',
 'http://forums.edmunds.com/discussions/tagged/x/sedans?Page=p3&amp;',
 'http://forums.edmunds.com/discussions/tagged/x/sedans?Page=p4&amp;',
 'http://forums.edmunds.com/discussions/tagged/x/sedans?Page=p5&amp;',
 'http://forums.edmunds.com/discussions/tagged/x/sedans?Page=p6&amp;',
 'http://forums.edmunds.com/discussions/tagged/x/sedans?Page=p7&amp;',
 'http://forums.edmunds.com/discussions/tagged/x/sedans?Page=p8&amp;amp%3B=',
 'http://forums.edmunds.com/discussions/tagged/x/sedans?Page=p9&amp;amp%3B=',
 'http://forums.edmunds.com/discussions/tagged/x/sedans?Page=p10&amp;amp%3B=',
 'http://forums.edmunds.com/discussions/tagged/x/sedans?Page=p11&amp;amp%3Bamp%3B=',
 'http://forums.edmunds.com/discussions/tagged/x/sedans?Page=p12&amp;amp%3Bamp%3B=',
 'http://forums.edmunds.com/discussions/tagged/x/sedans?Page=p13&amp;amp%3Bamp%3B=',
 'http://foru

In [46]:
BASE_URL = 'http://forums.edmunds.com'
page = urllib2.urlopen('http://forums.edmunds.com/discussions/tagged/x/sedans').read()
soup = BeautifulSoup(page, 'lxml')
Edmunds = soup.find("div", "PageControls Bottom")
pages = Edmunds.findAll("a")
links = re.findall('''href=["'](.[^"']+)["']''', str(pages))
for link in links:
    parent_links2.append(BASE_URL + link)


get_post_links function:  
this function gets discussion links from the edmunds sedan forum url 

In [84]:
#this function gets discussion links from the edmunds sedan forum url 

def get_post_links(url):
    #base_url = 'http://forums.edmunds.com/discussions/tagged/x/sedans'
    page = urllib2.urlopen(url).read()
    soup = BeautifulSoup(page, 'lxml')
    EdmundsPosts = soup.find("ul", "DataList Discussions")
    discussion_links = []
    for discussion in EdmundsPosts.findAll("div", "Title"):
        discussion_links.append(re.findall('''a href=["'](.[^"']+)["']''', str(discussion))[0])
    return discussion_links

In [61]:
a = [1,2,3,4,5,6]
a.extend([2,3,6])
a[6:8]

[2, 3]

Output of get_post_links for page 1 of the edmunds forum:

In [14]:
discussion_links = get_post_links(url)
discussion_links

['http://forums.edmunds.com/discussion/5844/chevrolet/cavalier/chevrolet-cavalier-electrical-system-questions',
 'http://forums.edmunds.com/discussion/7526/hyundai/malibu/midsize-sedans-2-0',
 'http://forums.edmunds.com/discussion/6369/jaguar/s-type/jaguar-s-type-transmission-problems',
 'http://forums.edmunds.com/discussion/5191/volkswagen/passat/volkswagen-passat-real-world-mpg-numbers',
 'http://forums.edmunds.com/discussion/31869/toyota/corolla/2015-toyota-corolla-prices-paid',
 'http://forums.edmunds.com/discussion/5270/oldsmobile/alero/oldsmobile-alero-starting-stalling-problems',
 'http://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans',
 'http://forums.edmunds.com/discussion/7037/bmw/5-series/bmw-5-series-starting-problems',
 'http://forums.edmunds.com/discussion/8842/buick/century/buick-century-electrical-lights',
 'http://forums.edmunds.com/discussion/1435/jaguar/x-type/jaguar-x-type',
 'http://forums.edmunds.com/discussion/13993/hyundai/ela

get_comments enters a URL and finds all text comments on the first page of comments  
Example output is shown below

In [83]:
def get_comments(discussion_url):
    page = urllib2.urlopen(discussion_url).read()
    soup = BeautifulSoup(page, 'lxml')
    soup = soup.find("div", "Column ContentColumn")
    post_list = soup.find_all("div", "Item-Body")
    comments = []
    for post in post_list:
        comment = post.find("div", "Message").get_text().strip().replace(',', '')
        #remove all commas.
        comments.append(comment)
    return comments

In [15]:
comments = get_comments('http://forums.edmunds.com/discussion/5844/chevrolet/cavalier/chevrolet-cavalier-electrical-system-questions')
','.join(comments)

u'I have a 96 chevy Cavalier. The horn went on by itself when no one was in car. I had to disconnect horn from the wire to it. Now the airbag cover where I hit to activate the horn clicks alot. I think the hornpad or relay is bad. Does anyone know anything about this or how to fix it.,I have the basic Cavalier standard shift etc.. The basic am/fm cd radio.The problem I can\'t set the clock never could !  Read the manual and I don\'t have the button they say I do on my radio.Before the clock was right 1/2 the year here on the East Coast.  Today I had a new battery put in and now it\'s totally not close to the time of day and now never will be Anyone know how to set the clock on a 2002 Cavalier ???Thanks much tommy,HiI bought a 2000 Cavalier for my teenage daughter this year.  It ran great thus far until Monday morning when she called me at work frantically and stated her car won\'t start.  Needless to say she missed her school on Monday.  Anyway I helped her to take the battery to the n

#Main Function
The Next step is to combine these functions in a main function to access text from all discussion links:

In [85]:

all_comments = ''
#iterate through pages 1 through n:
for page_i in links:
    discussion_links = get_post_links(page_i)
    #iterate through each discussion on page i:
    for link in discussion_links:
        comments = get_comments(link)
        all_comments += (','.join(comments)) + ','
        time.sleep(.5)

KeyboardInterrupt: 

Now we write all the comments stored in all_comments to a file, Edmunds_sedans1.csv  
We still need to access multiple pages of each discussion. right now it's only pulling from page 1 but that shouldn't be too hard. It's just a matter of getting the link for the next page, which will be easy to find in the html elements.

In [87]:
csvFile = open('Edmunds_sedans.csv', 'w')
csvFile.write(all_comments.encode('utf-8'))
csvFile.close()

In [15]:
csvFile = open('Edmunds_sedans.csv', 'r')
cleanup = csvFile.read()
csvFile.close()
cleanup = re.sub('\n', '', cleanup)
cleanup = re.sub(',(,)+', '', cleanup)
cleanup = re.sub(',',',\n', cleanup)
csvFile = open('Edmunds_sedans_cleaned.csv', 'w')
csvFile.write(cleanup)
csvFile.close()